In [1]:
!pwd

/data/guest_users/liangdong/liangdong/practice_demo/mxnet_CPM/pose_io


In [2]:
import sys
sys.path.append("/data/guest_users/liangdong/liangdong/practice_demo")

from pycocotools.coco import COCO
from collections import namedtuple
import numpy as np
from numpy import ma
import math
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib
import pylab
import mxnet as mx
pylab.rcParams['figure.figsize'] = (10.0, 8.0)


path1 = '/data/datasets/COCO/person_keypoints_trainval2014/person_keypoints_train2014.json'

annFile = path1
coco = COCO(annFile)
cats = coco.loadCats(coco.getCatIds())
catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds )
len(imgIds)

loading annotations into memory...
Done (t=5.38s)
creating index...
index created!


45174

In [3]:
trainimagepath = '/data/datasets/COCO/train2014/' 

import sys
sys.path.append("/data/guest_users/liangdong/liangdong/practice_demo")

from pycocotools.coco import COCO
from collections import namedtuple
import numpy as np
from numpy import ma
import math
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib
import pylab
import mxnet as mx
pylab.rcParams['figure.figsize'] = (10.0, 8.0)


path1 = '/data/datasets/COCO/person_keypoints_trainval2014/person_keypoints_train2014.json'

annFile = path1
coco = COCO(annFile)
cats = coco.loadCats(coco.getCatIds())
catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds )
len(imgIds)

loading annotations into memory...
Done (t=6.51s)
creating index...
index created!


45174

In [4]:
jointall=[]

In [5]:
from numpy import linalg as LA

In [6]:
trainimagepath = '/data/datasets/COCO/train2014/' 

### python read and write json

In [7]:
import json
import time

In [8]:
#import time

startt = time.time()
count = 1

#keysl = list(range(4,5))
joint_all = dict()
#joint_all[0] = [5]
#joint_all

In [9]:
numimgs = len(imgIds)

In [10]:
for i in range(0, numimgs):
    if(i%10==0):
        print i
    #print('----image: '+str(imgIds[i])+' ---------------')
    
    img = coco.loadImgs(imgIds[i])[0]
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    
    height = img['height']
    width = img['width']
    #print (height, width)
    prev_center = list(list())
    
    segmentations = list()
    
    for j in range(len(anns)):
        if anns[j]['num_keypoints']<5 or anns[j]['area']<32*32:
            segmentations.append(anns[j]['segmentation'])
            
    for j in range(len(anns)):
        
        if anns[j]['num_keypoints']<5 or anns[j]['area']<32*32:
            continue
        
        person_center = [anns[j]['bbox'][0] + anns[j]['bbox'][2]/2,
                         anns[j]['bbox'][0] + anns[j]['bbox'][3]/2]
        flag = 0
        isValidation = 0 
        
        for k in range(len(prev_center)):
            dist1 = prev_center[k][0] - person_center[0]
            dist2 = prev_center[k][1] - person_center[1]
            #print dist1, dist2
            if dist1*dist1+dist2*dist2 < prev_center[k][2]*0.3:
                flag = 1
                continue
                
        currentjoin={'isValidation': isValidation,
                     'img_paths': trainimagepath + img['file_name'],
                     'objpos': person_center,
                     'image_id': img['id'],
                     'bbox': anns[j]['bbox'],
                     'img_width': width,
                     'img_height': height,
                     'segment_area': anns[j]['area'],
                     'num_keypoints': anns[j]['num_keypoints'],
                     'joint_self': np.zeros((17,3)).tolist(),
                     'scale_provided': anns[j]['bbox'][3]/368.0,
                     'segmentations': segmentations,
                     'joint_others': {},
                     'annolist_index': i ,
                     'people_index': j,
                     'numOtherPeople':0,
                     'scale_provided_other':{},
                     'objpos_other':{},
                     'bbox_other':{},
                     'segment_area_other':{},
                     'num_keypoints_other':{}
                    }    
        
        '''
        currentjoin['isValidation'] = isValidation  
        currentjoin['img_paths'] = trainimagepath + img['file_name']
        currentjoin['objpos'] = person_center   
        currentjoin['image_id'] = img['id']
        currentjoin['bbox'] = anns[j]['bbox']
        currentjoin['segment_area'] = anns[j]['area']  
        currentjoin['num_keypoints'] = anns[j]['num_keypoints'] 
        anno = anns[j]['keypoints']
        
        currentjoin['joint_self'] = np.zeros((17,3))
        '''
        #print currentjoin['joint_self']
        
        #currentjoin['joint_self'][0,0]
        #print anns[j]
        for part in range(17):
            currentjoin['joint_self'][part][0] = anns[j]['keypoints'][part*3]
            currentjoin['joint_self'][part][1] = anns[j]['keypoints'][part*3+1]
            
            if(anns[j]['keypoints'][part*3+2] == 2):
                currentjoin['joint_self'][part][2] = 1
            elif(anns[j]['keypoints'][part*3+2] == 1):
                currentjoin['joint_self'][part][2] = 0                
            else:
                currentjoin['joint_self'][part][2] = 2
            
        # currentjoin['scale_provided'] = ann['bbox'][3]/368
        
        count_other = 1
        
        currentjoin['joint_others'] ={}
       
        for k in range(len(anns)):
            
            if k==j or anns[k]['num_keypoints']==0:
                continue
                
            annop = anns[k]
            
            currentjoin['scale_provided_other'][count_other] = annop['bbox'][3]/368
            currentjoin['objpos_other'][count_other] = [annop['bbox'][0]+annop['bbox'][2]/2, 
                                        annop['bbox'][1]+annop['bbox'][3]/2]
            currentjoin['bbox_other'][count_other] = annop['bbox']
            currentjoin['segment_area_other'][count_other] = annop['area']
            currentjoin['num_keypoints_other'][count_other] = annop['num_keypoints']
            
            currentjoin['joint_others'][count_other] = np.zeros((17,3)).tolist()
            
            for part in range(17):
                currentjoin['joint_others'][count_other][part][0] = annop['keypoints'][part*3]
                currentjoin['joint_others'][count_other][part][1] = annop['keypoints'][part*3+1]
                
                if(annop['keypoints'][part*3+2] == 2):
                    currentjoin['joint_others'][count_other][part][2] = 1
                elif(annop['keypoints'][part*3+2] == 1):
                    currentjoin['joint_others'][count_other][part][2] = 0
                else:
                    currentjoin['joint_others'][count_other][part][2] = 2
                
              
            currentjoin['numOtherPeople'] = len(currentjoin['joint_others'])
            # joint_all(count).annolist_index = i;
            # joint_all(count).people_index = p;
            # joint_all(count).numOtherPeople = length(joint_all(count).joint_others);
                    
            count_other = count_other + 1
                        
        prev_center.append([person_center[0], person_center[1],
                            max(anns[j]['bbox'][2], anns[j]['bbox'][3])])
        
    joint_all[i] = currentjoin
        #joint_all.append(currentjoin)
            
    #print('----image: '+str(imgIds[i])+' ---------------')
    
    count = count + 1
endt = time.time()
print (endt-startt)*45174/4

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

15700
15710
15720
15730
15740
15750
15760
15770
15780
15790
15800
15810
15820
15830
15840
15850
15860
15870
15880
15890
15900
15910
15920
15930
15940
15950
15960
15970
15980
15990
16000
16010
16020
16030
16040
16050
16060
16070
16080
16090
16100
16110
16120
16130
16140
16150
16160
16170
16180
16190
16200
16210
16220
16230
16240
16250
16260
16270
16280
16290
16300
16310
16320
16330
16340
16350
16360
16370
16380
16390
16400
16410
16420
16430
16440
16450
16460
16470
16480
16490
16500
16510
16520
16530
16540
16550
16560
16570
16580
16590
16600
16610
16620
16630
16640
16650
16660
16670
16680
16690
16700
16710
16720
16730
16740
16750
16760
16770
16780
16790
16800
16810
16820
16830
16840
16850
16860
16870
16880
16890
16900
16910
16920
16930
16940
16950
16960
16970
16980
16990
17000
17010
17020
17030
17040
17050
17060
17070
17080
17090
17100
17110
17120
17130
17140
17150
17160
17170
17180
17190
17200
17210
17220
17230
17240
17250
17260
17270
17280
17290
17300
17310
17320
17330
17340
17350
1736

29830
29840
29850
29860
29870
29880
29890
29900
29910
29920
29930
29940
29950
29960
29970
29980
29990
30000
30010
30020
30030
30040
30050
30060
30070
30080
30090
30100
30110
30120
30130
30140
30150
30160
30170
30180
30190
30200
30210
30220
30230
30240
30250
30260
30270
30280
30290
30300
30310
30320
30330
30340
30350
30360
30370
30380
30390
30400
30410
30420
30430
30440
30450
30460
30470
30480
30490
30500
30510
30520
30530
30540
30550
30560
30570
30580
30590
30600
30610
30620
30630
30640
30650
30660
30670
30680
30690
30700
30710
30720
30730
30740
30750
30760
30770
30780
30790
30800
30810
30820
30830
30840
30850
30860
30870
30880
30890
30900
30910
30920
30930
30940
30950
30960
30970
30980
30990
31000
31010
31020
31030
31040
31050
31060
31070
31080
31090
31100
31110
31120
31130
31140
31150
31160
31170
31180
31190
31200
31210
31220
31230
31240
31250
31260
31270
31280
31290
31300
31310
31320
31330
31340
31350
31360
31370
31380
31390
31400
31410
31420
31430
31440
31450
31460
31470
31480
3149

43680
43690
43700
43710
43720
43730
43740
43750
43760
43770
43780
43790
43800
43810
43820
43830
43840
43850
43860
43870
43880
43890
43900
43910
43920
43930
43940
43950
43960
43970
43980
43990
44000
44010
44020
44030
44040
44050
44060
44070
44080
44090
44100
44110
44120
44130
44140
44150
44160
44170
44180
44190
44200
44210
44220
44230
44240
44250
44260
44270
44280
44290
44300
44310
44320
44330
44340
44350
44360
44370
44380
44390
44400
44410
44420
44430
44440
44450
44460
44470
44480
44490
44500
44510
44520
44530
44540
44550
44560
44570
44580
44590
44600
44610
44620
44630
44640
44650
44660
44670
44680
44690
44700
44710
44720
44730
44740
44750
44760
44770
44780
44790
44800
44810
44820
44830
44840
44850
44860
44870
44880
44890
44900
44910
44920
44930
44940
44950
44960
44970
44980
44990
45000
45010
45020
45030
45040
45050
45060
45070
45080
45090
45100
45110
45120
45130
45140
45150
45160
45170
229574.799214


In [11]:
len(joint_all)

45174

In [12]:
import json
with open('data.json', 'w') as f:
     json.dump(joint_all, f)

In [13]:
# Reading data back
with open('data.json', 'r') as f:
     data = json.load(f)

In [14]:
data.keys()

[u'44884',
 u'11542',
 u'11543',
 u'11540',
 u'11541',
 u'11546',
 u'11547',
 u'11544',
 u'11545',
 u'11548',
 u'11549',
 u'5988',
 u'5989',
 u'19399',
 u'19398',
 u'5982',
 u'5983',
 u'5980',
 u'5981',
 u'5986',
 u'5987',
 u'5984',
 u'5985',
 u'16643',
 u'29237',
 u'270',
 u'271',
 u'272',
 u'273',
 u'274',
 u'275',
 u'276',
 u'277',
 u'278',
 u'279',
 u'16707',
 u'16706',
 u'16701',
 u'16700',
 u'16703',
 u'16702',
 u'12019',
 u'12018',
 u'12015',
 u'12014',
 u'12017',
 u'12016',
 u'12011',
 u'12010',
 u'12013',
 u'12012',
 u'17258',
 u'17259',
 u'17250',
 u'17251',
 u'17252',
 u'17253',
 u'17254',
 u'17255',
 u'17256',
 u'17257',
 u'10709',
 u'10708',
 u'9258',
 u'9259',
 u'9256',
 u'9257',
 u'9254',
 u'9255',
 u'9252',
 u'9253',
 u'9250',
 u'9251',
 u'21619',
 u'21618',
 u'21615',
 u'21614',
 u'21617',
 u'21616',
 u'21611',
 u'21610',
 u'21613',
 u'21612',
 u'19397',
 u'19396',
 u'19395',
 u'44799',
 u'44798',
 u'19394',
 u'44791',
 u'19393',
 u'44793',
 u'44792',
 u'44795',
 u'447

In [15]:
data['44884']['scale_provided']

0.31755434782608694

In [16]:
data['11542']['scale_provided']

0.3053804347826087

In [17]:
data['395']

{u'annolist_index': 395,
 u'bbox': [0, 0, 444.9, 333],
 u'bbox_other': {},
 u'image_id': 132147,
 u'img_height': 333,
 u'img_paths': u'/data/datasets/COCO/train2014/COCO_train2014_000000132147.jpg',
 u'img_width': 500,
 u'isValidation': 0,
 u'joint_others': {},
 u'joint_self': [[261, 134, 1],
  [301, 157, 1],
  [277, 109, 1],
  [320, 245, 1],
  [0, 0, 2],
  [207, 272, 1],
  [189, 125, 1],
  [0, 0, 2],
  [114, 78, 1],
  [154, 256, 1],
  [62, 11, 1],
  [0, 0, 2],
  [0, 0, 2],
  [0, 0, 2],
  [31, 81, 1],
  [0, 0, 2],
  [0, 0, 2]],
 u'numOtherPeople': 0,
 u'num_keypoints': 10,
 u'num_keypoints_other': {},
 u'objpos': [222.45, 166],
 u'objpos_other': {},
 u'people_index': 0,
 u'scale_provided': 0.904891304347826,
 u'scale_provided_other': {},
 u'segment_area': 97570.6352,
 u'segment_area_other': {},
 u'segmentations': []}